### Step1. import library

In [1]:
import traitlets , time
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

### Display live camera feed

So let's get started.  First, let's initialize and display our camera like we did in the *teleoperation* notebook.  

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task).
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [2]:
camera = Camera.instance(capture_flip = 2,width=224, height=224 , capture_device=1)

image = widgets.Image(format='jpeg', width=224, height=224 )  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Awesome, next let's create a few directories where we'll store all our data. We'll create a folder dataset that will contain two sub-folders free and blocked, where we'll place the images for each scenario.

In [3]:
import os

item = ['free' , 'green_light' , 'red_light' , 'parking' , 'zebra' , 'fence_open' , 'fence_close' , 'duck']
dataset_dir = {}
for name in item:
    dataset_dir[name]= 'dataset/' + str( item.index(name) ) + '_' + name
    print(dataset_dir[name])
# we have this "try/except" statement because these next functions can throw an error if the directories exist already

try:
    for name in dataset_dir.keys() :
        os.makedirs(dataset_dir[name])
except FileExistsError:
    print('Directories not created becasue they already exist')

dataset/0_free
dataset/1_green_light
dataset/2_red_light
dataset/3_parking
dataset/4_zebra
dataset/5_fence_open
dataset/6_fence_close
dataset/7_duck


If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about as many images.  It also helps to know how many images we've collected overall.

In [4]:
button_layout = widgets.Layout(width='128px', height='64px')
button_style = ['primary', 'success', 'info', 'warning', 'danger', '']

button = {}
count  = {}

for name in item:
    button[ name ] = widgets.Button(description= name, button_style='success', layout=button_layout)
    count[ name ]  = widgets.IntText(layout=button_layout, value=len(os.listdir( dataset_dir[name])))
    display(widgets.HBox([ button[ name], count[name] ] ) ) 
    button[ name ].button_style = button_style[ item.index(name) % 6 ]

## you can change the color of button
button[ 'green_light' ].button_style = 'primary' 
button[ 'green_light' ].button_style = 'success'    
button[ 'red_light' ].button_style = 'danger'
button[ 'parking' ].button_style = 'info'

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

If you want to test something , you can use it : 

test = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
    )

In [5]:
from uuid import uuid1

class save_image():
    def __init__(self,item , dataset_dir):
        self.item = item
        self.dataset_dir = dataset_dir

    def save_name(self , kind_of_item):
        self.save_snapshot(kind_of_item)
        count[ kind_of_item ].value = len(os.listdir( dataset_dir[kind_of_item]))
            
    def save_snapshot(self,name):
        global image_path
        image_path = os.path.join(self.dataset_dir[name], str(uuid1()) + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image.value)    

            

# Notice
Now , you can choose keyboard or joystick to teleop Jetbot to collected data.
But remember , you just only can select one to do.

### Use Keyboard 

In [6]:
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
# If you have four serval items to do , please add :  button[item[number]].on_click(lambda x: button_reaction.save_name(item[number]) )
# number is your item index , start at 0 .
button_reaction = save_image(item,dataset_dir)
button[item[0]].on_click(lambda x: button_reaction.save_name(item[0]) ) # " controller_button['X'] "
button[item[1]].on_click(lambda x: button_reaction.save_name(item[1]) ) # " controller_button['Y'] "
button[item[2]].on_click(lambda x: button_reaction.save_name(item[2]) ) # " controller_button['B'] "
button[item[3]].on_click(lambda x: button_reaction.save_name(item[3]) ) # " controller_button['A'] "
button[item[4]].on_click(lambda x: button_reaction.save_name(item[4]) ) # " controller_button['L2'] "
button[item[5]].on_click(lambda x: button_reaction.save_name(item[5]) ) # " controller_button['R2'] "
button[item[6]].on_click(lambda x: button_reaction.save_name(item[6]) ) # " controller_button['start'] "
button[item[7]].on_click(lambda x: button_reaction.save_name(item[7]) ) # " controller_button['select'] "

Second is creating slider , link controller and slider.
If you don't use controlle , please do not run below code.

If you use controller , please run below code.

# Start Collect data!

In [11]:
display(widgets.HBox([image]))

for text in item:
    display(widgets.HBox([button[text] , count[text]]))

Sometimes the folder will has checkpoint of ipython , we need delete it .

In [12]:
!rm -rf dataset/0_free/.ipynb_checkpoints
!rm -rf dataset/1_green_light/.ipynb_checkpoints
!rm -rf dataset/2_red_light/.ipynb_checkpoints
!rm -rf dataset/3_parking/.ipynb_checkpoints
!rm -rf dataset/4_zebra/.ipynb_checkpoints
!rm -rf dataset/5_fence_open/.ipynb_checkpoints
!rm -rf dataset/6_fence_close/.ipynb_checkpoints
!rm -rf dataset/7_duck/.ipynb_checkpoints


Before shutdown this code , will need to unobserve camera 

In [15]:
camera.running = False